In [ ]:
from abc import ABC, abstractmethod
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import os

######## 수정 금지 #########
class BaseCrawler(ABC):
    def __init__(self, output_dir: str):
        self.output_dir = output_dir

    @abstractmethod
    def start_browser(self):
        pass

    @abstractmethod
    def navigate_and_click(self):
        pass

    @abstractmethod
    def scrape_page_content(self):
        pass

    @abstractmethod
    def save_to_database(self):
        pass
############################

# 아래부터 BaseCrawler를 상속하는 BookInfoCrawler 클래스입니다.
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from bs4 import BeautifulSoup
import pandas as pd

class BookInfoCrawler(BaseCrawler):
    def __init__(self, output_dir: str):
        super().__init__(output_dir)
        # 실제 크롤링할 도서 페이지 URL (예시: Goodreads의 Sapiens 도서 페이지)
        self.base_url = "https://www.goodreads.com/book/show/23692271-sapiens"
        self.driver = None
        self.book_info: dict = {}
        # ChromeDriver 경로 (반드시 실제 파일 위치와 일치해야 합니다)
        self.webdriver_path = r"C:\Users\user\.cache\selenium\chromedriver\win64\131.0.6778.264\chromedriver.exe"
        # 추출할 정보에 대한 CSS Selector (실제 페이지 구조에 맞게 수정 필요)
        self.selectors = {
            'book_name': "h1#bookTitle",               # 예시: 책 제목 선택자
            'book_image': "img#coverImage",             # 예시: 책 표지 이미지 선택자
            'author': "a.authorName > span",            # 예시: 저자 이름 선택자
            'total_star': "span[itemprop='ratingValue']", # 예시: 평균 별점 선택자
            'published': "div#details div.row",         # 예시: 출판 정보 (세부 항목, 필요시 추가 가공)
            'pages': "span[itemprop='numberOfPages']"     # 예시: 페이지 수 선택자
        }
        self.xpath = {
            'book_name': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[1]/div[1]/h1',
            'book_image': '//*[@id="__next"]/div[2]/main/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/img',
            'author': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[1]/h3/div/span[1]/a/span',
            'total_star': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[2]/a/div[1]/div',
            'published': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'pages': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'description': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[4]/div/div[1]/div/div/span'
        }

    def start_browser(self):
        """
        Selenium WebDriver(Chrome) 객체를 생성하고 준비하는 메서드.
        """
        options = Options()
        # 필요 시 headless 옵션 활성화 (화면 표시 없이 실행)
        # options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
        options.add_argument("--lang=ko_KR")
        options.add_argument("--charset=utf-8")
        
        self.driver = webdriver.Chrome(
            service=Service(self.webdriver_path),
            options=options
        )


    def navigate_and_click(self):
        """
        1) base_url로 이동한 후 페이지가 로딩되면,  
        2) 로그인 창 외 빈 영역(지정된 요소)을 클릭하여 페이지를 준비합니다.
        """
        if self.driver is None:
            raise RuntimeError("start_browser()를 먼저 호출해야 합니다.")
        
        self.driver.get(self.base_url)
        time.sleep(3)  # 페이지 로딩 대기
        

        try:
            # 지정된 요소가 클릭 가능할 때까지 대기한 후 클릭
            element = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[1]/div[1]/h1')
                )
            )
            element.click()
            time.sleep(1)  # 클릭 후 페이지 반영 대기
        except Exception as e:
            print("빈 영역 클릭 중 오류 발생:", e)

    def scrape_page_content(self):
        """
        준비된 페이지에서 도서 정보를 추출하는 메서드.
        추출 항목:
        - book_name
        - book_image (이미지 URL)
        - author
        - total_star
        - published
        - pages
        - description
        """
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, self.xpath['book_name']))
            )
        except Exception as e:
            print("페이지 로딩 중 오류 발생:", e)
    
        self.book_info = {}
        for key, xpath in self.xpath.items():
            try:
                element = self.driver.find_element(By.XPATH, xpath)
                if key == 'book_image':
                    # 이미지의 경우 src 속성에서 URL을 추출합니다.
                    self.book_info[key] = element.get_attribute("src")
                else:
                    self.book_info[key] = element.text.strip()
            except Exception as e:
                print(f"'{key}' 요소 추출 중 오류 발생: {e}")
                self.book_info[key] = None


                
    def save_to_database(self):
        """
        추출한 도서 정보를 CSV 파일로 저장하는 메서드.
        (BaseCrawler의 추상 메서드 save_to_database()를 구현합니다.)
        """
        if not self.book_info:
            print("저장할 데이터가 없습니다. 먼저 scrape_reviews()를 실행하세요.")
            return
        
        df = pd.DataFrame([self.book_info])
        output_path = os.path.join(self.output_dir, "book_info.csv")
        #df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print("도서 정보가 저장되었습니다:", output_path)
        self.data = df
    
    def close_browser(self):
        """
        Selenium WebDriver 종료 메서드.
        """
        if self.driver:
            self.driver.quit()


# 사용 예시
if __name__ == "__main__":
    output_directory = "./output" 
    # 결과를 저장할 폴더가 없으면 생성

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    crawler = BookInfoCrawler(output_directory)
    crawler.start_browser()
    crawler.navigate_and_click()
    crawler.scrape_page_content()
    crawler.save_to_database()
    #crawler.close_browser()


도서 정보가 저장되었습니다: ./output\book_info.csv


In [26]:
crawler.data

,book_name,book_image,author,total_star,published,pages
0,Sapiens: A Brief History of Humankind,https://images-na.ssl-images-amazon.com/images...,Yuval Noah Harari,4.35,"First published January 1, 2011","First published January 1, 2011"


In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import os
import pandas as pd

class BookInfoCrawler:
    def __init__(self, output_dir: str):
        self.output_dir = output_dir
        # 기본 도서 상세 페이지 URL (테스트용)
        self.base_url = "https://www.goodreads.com/book/show/23692271-sapiens"
        self.driver = None
        self.book_info: dict = {}
        self.output = None 
        # ChromeDriver 경로 (실제 파일 위치로 수정)
        self.webdriver_path = r"C:\Users\user\.cache\selenium\chromedriver\win64\131.0.6778.264\chromedriver.exe"

        # 도서 상세 페이지에서 필요한 정보 추출용 XPath (실제 페이지에 맞게 수정)
        self.xpath = {
            'book_name': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[1]/div[1]/h1',
            'book_image': '//*[@id="__next"]/div[2]/main/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/img',
            'author': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[1]/h3/div/span[1]/a/span',
            'total_star': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[2]/a/div[1]/div',
            'published': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'pages': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'description': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[4]/div/div[1]/div/div/span'
        }
        
        # [scrap_list] 관련 변수: 책 리스트 페이지 URL 및 책 링크 요소 XPath (실제 값으로 수정)
        self.book_list_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0"
        self.book_link_xpath = '//*[@id="all_votes"]/table/tbody/tr[1]/td[3]/a'
        
        # [scrap_book] 내 'view more reviews' 버튼 XPath (실제 값으로 수정)
        self.view_more_reviews_button_xpath = '//*[@id="ReviewsSection"]/div[6]/div[4]/a'
        
        # [scrap_reviews] 관련 변수 (실제 값으로 수정)
        self.filter_button_xpath = '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[4]/div[1]/div[2]/div/button'
        # (기존 review_item_xpath는 사용하지 않고, 리뷰 컨테이너별로 하위 요소를 추출할 예정)
        self.show_more_button_xpath = "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button"
        
        # 영어 선택 관련 XPath (수정된 값)
        self.english_label_xpath = '/html/body/div[3]/div/div[2]/span/div/div[6]/div[3]/label/span'
        # Apply 버튼의 xpath
        self.apply_button_xpath = '/html/body/div[3]/div/div[3]/div[2]/button'
        
        # 로그인 오버레이 닫기 버튼의 CSS 셀렉터 (오버레이 닫기 버튼)
        self.login_overlay_dismiss_selector = "body > div.Overlay.Overlay--floating > div > div.Overlay__header > div > div > button"
        
        # 리뷰 내부 요소 xpath 템플릿 (INDEX를 실제 리뷰 컨테이너 번호로 대체할 것)
        self.review_xpaths = {
            "reviewer": "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/div/div/section[2]/span[1]/div/a",
            "date":     "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/span/a",
            "rating":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/div/span",
            "review":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[2]/section/div/div[1]/span"
        }
    
    def start_browser(self):
        """
        Selenium WebDriver(Chrome) 객체를 생성하여 브라우저를 시작합니다.
        """
        options = Options()
        # 필요에 따라 headless 모드 사용 가능
        # options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
        options.add_argument("--lang=ko_KR")
        options.add_argument("--charset=utf-8")
        
        self.driver = webdriver.Chrome(
            service=Service(self.webdriver_path),
            options=options
        )
    
    def check_and_dismiss_overlay(self, timeout=5):
        """
        주어진 timeout(초) 동안 로그인 오버레이 닫기 버튼이 나타나면 클릭하여 해제합니다.
        오버레이가 없으면 그냥 넘어갑니다.
        """
        end_time = time.time() + timeout
        dismissed = False
        while time.time() < end_time:
            try:
                dismiss_button = self.driver.find_element(By.CSS_SELECTOR, self.login_overlay_dismiss_selector)
                if dismiss_button.is_displayed() and dismiss_button.is_enabled():
                    dismiss_button.click()
                    print("로그인 오버레이가 해제되었습니다.")
                    dismissed = True
                    break
            except Exception:
                # 버튼이 없으면 오버레이가 없는 것으로 간주
                break
            time.sleep(0.5)
        return dismissed

    def scrap_book(self):
        """
        1. 도서 상세 페이지에서 책 정보를 추출합니다.
        2. 'view more reviews' 버튼을 클릭하기 전에 오버레이를 확인 및 제거합니다.
        3. 'view more reviews' 버튼을 클릭하여 리뷰 페이지로 이동합니다.
        4. scrap_reviews()를 호출하여 리뷰(영어 필터 적용) 데이터를 수집합니다.
        5. 책 정보와 리뷰를 포함한 딕셔너리를 반환합니다.
        """
        # 1. 책 정보 추출
        self.book_info = {}
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, self.xpath['book_name']))
            )
        except Exception as e:
            print("책 정보 로딩 중 오류 발생:", e)
            
        for key, xpath_val in self.xpath.items():
            try:
                element = self.driver.find_element(By.XPATH, xpath_val)
                if key == 'book_image':
                    self.book_info[key] = element.get_attribute("src")
                else:
                    self.book_info[key] = element.text.strip()
            except Exception as e:
                print(f"{key} 정보 추출 중 오류 발생: {e}")
                self.book_info[key] = None

        # 2. 오버레이 확인 및 제거
        self.check_and_dismiss_overlay(timeout=5)
        
        # 3. 'view more reviews' 버튼 클릭하여 리뷰 페이지로 이동
        try:
            view_more_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.view_more_reviews_button_xpath))
            )
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.Overlay.Overlay--floating"))
            )
            view_more_button.click()
            time.sleep(2)
        except Exception as e:
            print("리뷰 페이지로 이동하는 'view more reviews' 버튼 클릭 중 오류 발생:", e)
            self.book_info['reviews'] = []
            return self.book_info
        
        # 4. 리뷰 추출 (영어 필터 적용)
        reviews = self.scrap_reviews(target_review_count=10)
        self.book_info['reviews'] = reviews
        
        return self.book_info

    def scrap_reviews(self, target_review_count=10):
        """
        리뷰 컨테이너는
          //*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]
        에서 INDEX를 대입하여 접근합니다.
          - 단, div[2]는 광고이므로 건너뜁니다.
          - 예를 들어, 유효한 리뷰 컨테이너는 div[1], div[3], div[4], …, div[63]까지 있을 수 있습니다.
          - 만약 더 이상 해당 컨테이너가 없으면, "show more reviews" 버튼( xpath: 
            //*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button )을
            클릭하여 추가 리뷰를 로딩하고, 컨테이너 번호를 갱신합니다.
        각 리뷰 컨테이너 내에서, self.review_xpaths에 저장된 하위 xpath( reviewer, date, rating, review )
        를 사용하여 해당 리뷰의 정보를 추출합니다.
        만약 하위 요소(예: 리뷰어, 리뷰 텍스트 등)가 없으면 해당 컨테이너를 광고로 판단하고 건너뜁니다.
        target_review_count 만큼 리뷰를 수집하면 리스트를 반환합니다.
        """
        reviews = []
        current_index = 1  # 리뷰 컨테이너 번호 시작 (예: div[1])
        last_index_before_more = 63  # 이 번호 이후엔 "show more reviews" 버튼을 눌러야 함
        next_index_after_more = 94   # "show more reviews" 클릭 후, 새 컨테이너 시작 번호 예시

        while len(reviews) < target_review_count:
            container_xpath = f"//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[{current_index}]"
            try:
                container = self.driver.find_element(By.XPATH, container_xpath)
            except Exception:
                # 리뷰 컨테이너가 없으면 "show more reviews" 버튼 클릭 시도
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    current_index = next_index_after_more
                    continue
                except Exception as e:
                    print("show more reviews 버튼 클릭 실패:", e)
                    break

            # 건너뛰기: div[2]는 광고라 가정
            if current_index == 2:
                current_index += 1
                continue

            # 각 리뷰 컨테이너에서 하위 요소 추출 시도
            review_data = {}
            valid = True
            for key, xpath_template in self.review_xpaths.items():
                # 컨테이너 번호 자리에 current_index 삽입
                xpath = xpath_template.replace("INDEX", str(current_index))
                try:
                    element = self.driver.find_element(By.XPATH, xpath)
                    review_data[key] = element.text.strip()
                except Exception as e:
                    # 필수 요소를 찾지 못하면 광고일 가능성이 있으므로 건너뜀
                    valid = False
                    break
            if valid and review_data.get("review", ""):
                reviews.append(review_data)
                print(f"수집된 리뷰 {len(reviews)} (컨테이너 div[{current_index}]): {review_data}")
            if len(reviews) >= target_review_count:
                break

            current_index += 1
            # 만약 현재 인덱스가 last_index_before_more를 초과하면 "show more reviews" 버튼 클릭
            if current_index > last_index_before_more:
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    current_index = next_index_after_more
                except Exception as e:
                    print("더 이상 리뷰 로딩 불가 또는 show more reviews 클릭 중 오류 발생:", e)
                    break

        return reviews

    def scrap_list(self):
        """
        1. 책 리스트 페이지에 접속하여 책 링크들을 XPath로 추출합니다.
        2. 테스트용으로 첫 번째 책 링크를 선택해 해당 상세 페이지로 이동합니다.
        3. scrap_book()을 호출하여 책 정보와 리뷰를 수집합니다.
        """
        self.driver.get(self.book_list_url)
        time.sleep(3)
        
        try:
            book_links = self.driver.find_elements(By.XPATH, self.book_link_xpath)
            if not book_links:
                print("책 리스트 페이지에서 책 링크를 찾을 수 없습니다.")
                return None
            first_book_link = book_links[0].get_attribute("href")
        except Exception as e:
            print("책 링크 추출 중 오류 발생:", e)
            return None
        
        self.driver.get(first_book_link)
        time.sleep(3)
        
        # 페이지 이동 후 오버레이 확인 및 제거
        self.check_and_dismiss_overlay(timeout=5)
        
        book_data = self.scrap_book()
        self.output = book_data 
        return book_data

    def close_browser(self):
        if self.driver:
            self.driver.quit()


# __main__ 코드: 책 리스트에서 첫 번째 책을 선택 후, 책 정보와 리뷰(영어 필터 적용)가 수집됨.
if __name__ == "__main__":
    output_directory = "./output"  
    # 결과를 저장할 폴더가 없으면 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    crawler = BookInfoCrawler(output_directory)
    crawler.start_browser()
    
    # scrap_list()를 호출하면, 첫 번째 책의 상세 페이지로 이동하고,
    # 이후 scrap_book()과 scrap_reviews()를 통해 책 정보 및 리뷰가 수집됩니다.
    book_data = crawler.scrap_list()
    print("수집된 책 데이터:")
    print(book_data)
    
    crawler.close_browser()


수집된 리뷰 1 (컨테이너 div[1]): {'reviewer': 'Nataliya', 'date': 'April 25, 2023', 'rating': '', 'review': 'Suzanne Collins has balls ovaries of steel to make us willingly cheer for a teenage girl to kill other children. In a YA book.\nTwo reasons why this book rocks: (a) It is not Twilight, and (b) I really hate reality shows.\n\n\n\n\nYes, this book is full of imperfections. It often requires a strenuous suspension of disbelief. It can cause a painful amount of eye-rolling and shaking fist at the book pages. Its style is choppy and the first-person present tense gets annoying. The story is simple, and the message is heavy-handed. And here is an obligatory taken out of contest Twilight-bashing quote:\nBecause Katniss is cool and a badass. She is fierce, independent, resourceful, intelligent, and skilled. She is loyal to her friends and family. She is a survivor. She skewers that apple in the pig\'s mouth with an arrow in front of the Gamemakers in the most awesome way imaginable. For all that

In [50]:
import json

# Save crawler.output to a JSON file
with open('crawler_output.json', 'w', encoding='utf-8') as f:
    json.dump(crawler.output, f, ensure_ascii=False, indent=4)

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import os
import pandas as pd

class BookInfoCrawler:
    def __init__(self, output_dir: str):
        self.output_dir = output_dir
        # 기본 도서 상세 페이지 URL (테스트용)
        #self.base_url = "https://www.goodreads.com/book/show/23692271-sapiens"
        self.driver = None
        self.book_info: dict = {}
        # 최종 결과 저장용 변수 (필요 시 사용)
        self.output = None
        # ChromeDriver 경로 (실제 파일 위치로 수정)
        self.webdriver_path = r"C:\Users\user\.cache\selenium\chromedriver\win64\131.0.6778.264\chromedriver.exe"

        # 도서 상세 페이지에서 필요한 정보 추출용 XPath (실제 페이지에 맞게 수정)
        self.xpath = {
            'book_name': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[1]/div[1]/h1',
            'book_image': '//*[@id="__next"]/div[2]/main/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/img',
            'author': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[1]/h3/div/span[1]/a/span',
            'total_star': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[2]/a/div[1]/div',
            'published': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'pages': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'description': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[4]/div/div[1]/div/div/span'
        }
        
        # [scrap_list] 관련 변수: 책 리스트 페이지 URL 및 책 링크 요소 XPath (실제 값으로 수정)
        self.book_list_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0"
        self.book_link_xpath = '//*[@id="all_votes"]/table/tbody/tr[1]/td[3]/a'
        
        # [scrap_book] 내 'view more reviews' 버튼 XPath (실제 값으로 수정)
        self.view_more_reviews_button_xpath = '//*[@id="ReviewsSection"]/div[6]/div[4]/a'
        
        # [scrap_reviews] 관련 변수 (실제 값으로 수정)
        self.filter_button_xpath = '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[4]/div[1]/div[2]/div/button'
        # 리뷰 항목 xpath는 각 컨테이너 내에서 하위 요소 추출을 위해 사용됩니다.
        # show more reviews 버튼 xpath
        self.show_more_button_xpath = "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button"
        
        # 영어 선택 관련 XPath (수정된 값)
        self.english_label_xpath = '/html/body/div[3]/div/div[2]/span/div/div[6]/div[3]/label/span'
        # Apply 버튼의 xpath
        self.apply_button_xpath = '/html/body/div[3]/div/div[3]/div[2]/button'
        
        # 로그인 오버레이 닫기 버튼의 CSS 셀렉터 (오버레이 닫기 버튼)
        self.login_overlay_dismiss_selector = "body > div.Overlay.Overlay--floating > div > div.Overlay__header > div > div > button"
        
        # 리뷰 내부 요소 xpath 템플릿 (INDEX를 실제 리뷰 컨테이너 번호로 대체)
        self.review_xpaths = {
            "reviewer": "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/div/div/section[2]/span[1]/div/a",
            "date":     "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/span/a",
            "rating":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/div/span",
            "review":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[2]/section/div/div[1]/span"
        }
        
        # 책 리스트 페이지 관련 변수:
        self.book_list_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0"
        # 기본적으로 한 페이지에 100권의 책이 있다고 가정
        self.num_books = 100
        self.book_link_xpath = "//*[@id='all_votes']/table/tbody/tr[1]/td[3]/a"  # 기본 템플릿(행 번호는 개별 처리)
        # ...
    
    def start_browser(self):
        """
        Selenium WebDriver(Chrome) 객체를 생성하여 브라우저를 시작합니다.
        """
        options = Options()
        # 필요에 따라 headless 모드 사용 가능
        # options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
        options.add_argument("--lang=ko_KR")
        options.add_argument("--charset=utf-8")
        
        self.driver = webdriver.Chrome(
            service=Service(self.webdriver_path),
            options=options
        )
    
    def check_and_dismiss_overlay(self, timeout=5):
        """
        주어진 timeout(초) 동안 로그인 오버레이 닫기 버튼이 나타나면 클릭하여 해제합니다.
        오버레이가 없으면 그냥 넘어갑니다.
        """
        end_time = time.time() + timeout
        dismissed = False
        while time.time() < end_time:
            try:
                dismiss_button = self.driver.find_element(By.CSS_SELECTOR, self.login_overlay_dismiss_selector)
                if dismiss_button.is_displayed() and dismiss_button.is_enabled():
                    dismiss_button.click()
                    print("로그인 오버레이가 해제되었습니다.")
                    dismissed = True
                    break
            except Exception:
                break
            time.sleep(0.5)
        return dismissed

    def scrap_book(self):
        """
        1. 도서 상세 페이지에서 책 정보를 추출합니다.
        2. 'view more reviews' 버튼을 클릭하기 전에 오버레이를 확인 및 제거합니다.
        3. 'view more reviews' 버튼을 클릭하여 리뷰 페이지로 이동합니다.
        4. scrap_reviews()를 호출하여 리뷰(영어 필터 적용) 데이터를 수집합니다.
        5. 책 정보와 리뷰를 포함한 딕셔너리를 반환합니다.
        """
        # 1. 책 정보 추출
        self.book_info = {}
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, self.xpath['book_name']))
            )
        except Exception as e:
            print("책 정보 로딩 중 오류 발생:", e)
            
        for key, xpath_val in self.xpath.items():
            try:
                element = self.driver.find_element(By.XPATH, xpath_val)
                if key == 'book_image':
                    self.book_info[key] = element.get_attribute("src")
                else:
                    self.book_info[key] = element.text.strip()
            except Exception as e:
                print(f"{key} 정보 추출 중 오류 발생: {e}")
                self.book_info[key] = None

        # 2. 오버레이 확인 및 제거
        self.check_and_dismiss_overlay(timeout=5)
        
        # 3. 'view more reviews' 버튼 클릭하여 리뷰 페이지로 이동
        try:
            view_more_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.view_more_reviews_button_xpath))
            )
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.Overlay.Overlay--floating"))
            )
            view_more_button.click()
            time.sleep(2)
        except Exception as e:
            print("리뷰 페이지로 이동하는 'view more reviews' 버튼 클릭 중 오류 발생:", e)
            self.book_info['reviews'] = []
            return self.book_info
        
        # 4. 리뷰 추출 (영어 필터 적용)
        reviews = self.scrap_reviews(target_review_count=10)
        self.book_info['reviews'] = reviews
        
        return self.book_info

    def scrap_reviews(self, target_review_count=10):
        """
        리뷰 컨테이너는
          //*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]
        에서 INDEX를 대입하여 접근합니다.
          - 단, div[2]는 광고이므로 건너뜁니다.
          - 예를 들어, 유효한 리뷰 컨테이너는 div[1], div[3], div[4], …, div[63]까지 있을 수 있습니다.
          - 만약 더 이상 해당 컨테이너가 없으면, "show more reviews" 버튼(//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button)
            을 클릭하여 추가 리뷰를 로딩하고, 컨테이너 번호를 갱신합니다.
        각 리뷰 컨테이너 내에서, self.review_xpaths에 저장된 하위 xpath( reviewer, date, rating, review )
        를 사용하여 해당 리뷰의 정보를 추출합니다.
        만약 하위 요소(예: 리뷰어, 리뷰 텍스트 등)가 없으면 해당 컨테이너를 광고로 판단하고 건너뜁니다.
        target_review_count 만큼 리뷰를 수집하면 리스트를 반환합니다.
        """
        reviews = []
        current_index = 1  # 리뷰 컨테이너 번호 시작 (예: div[1])
        last_index_before_more = 63  # 이 번호 이후엔 "show more reviews" 버튼을 눌러야 함
        next_index_after_more = 94   # "show more reviews" 클릭 후, 새 컨테이너 시작 번호 예시
        
        # 컨테이너를 찾지 못하는 최대 시도 횟수 (무한 루프 방지)
        max_attempts = 5
        attempts = 0
        
        while len(reviews) < target_review_count and attempts < max_attempts:
            container_xpath = f"//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[{current_index}]"
            try:
                container = self.driver.find_element(By.XPATH, container_xpath)
                attempts = 0  # 컨테이너를 찾으면 시도 카운트를 리셋
            except Exception:
                attempts += 1
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    current_index = next_index_after_more
                    continue
                except Exception as e:
                    print("show more reviews 버튼 클릭 실패:", e)
                    break

            # 건너뛰기: div[2]는 광고라 가정
            if current_index == 2:
                current_index += 1
                continue

            # 각 리뷰 컨테이너 내에서 하위 요소 추출
            review_data = {}
            valid = True
            for key, xpath_template in self.review_xpaths.items():
                xpath = xpath_template.replace("INDEX", str(current_index))
                try:
                    element = self.driver.find_element(By.XPATH, xpath)
                    if key == "rating":
                        # rating은 aria-label 속성에서 추출
                        review_data[key] = element.get_attribute("aria-label").strip()
                    else:
                        review_data[key] = element.text.strip()
                except Exception as e:
                    valid = False
                    break
            if valid and review_data.get("review", ""):
                reviews.append(review_data)
                print(f"수집된 리뷰 {len(reviews)} (컨테이너 div[{current_index}]): {review_data}")
            if len(reviews) >= target_review_count:
                break

            current_index += 1
            # 만약 현재 인덱스가 last_index_before_more를 초과하면 "show more reviews" 버튼 클릭
            if current_index > last_index_before_more:
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    current_index = next_index_after_more
                except Exception as e:
                    print("더 이상 리뷰 로딩 불가 또는 show more reviews 클릭 중 오류 발생:", e)
                    break

        if attempts >= max_attempts:
            print("리뷰 컨테이너 로드 시도 횟수 초과, 리뷰 수집 종료")
        return reviews


    def scrap_list(self):
        """
        1. 책 리스트 페이지에 접속하여, tr[1]부터 tr[num_books]까지의 책 링크( href )를 수집합니다.
        2. 각 책 링크로 이동하여 scrap_book()을 호출해 책 정보와 리뷰를 수집합니다.
        3. 한 페이지의 모든 책을 처리한 후, "next" 버튼(//*[@id="all_votes"]/div[1]/a[12])을 클릭하여
           다음 페이지로 이동하고 같은 과정을 반복합니다.
        4. 최종적으로 모든 책의 데이터를 self.output 에 저장하고 반환합니다.
        """
        all_books = []
        current_page = 1
        
        while True:
            print(f"--- 책 리스트 페이지 {current_page} 처리 시작 ---")
            # 현재 페이지 URL로 이동 (페이지 번호가 URL에 반영되어 있지 않다면, next 버튼 클릭으로 변경된 URL를 그대로 사용)
            self.driver.get(self.book_list_url)
            time.sleep(3)
            
            # 한 페이지의 책 링크를 수집 (tr[1] ~ tr[num_books])
            page_links = []
            for i in range(1, self.num_books + 1):
                xpath_link = f"//*[@id='all_votes']/table/tbody/tr[{i}]/td[3]/a"
                try:
                    elem = self.driver.find_element(By.XPATH, xpath_link)
                    link = elem.get_attribute("href")
                    page_links.append(link)
                except Exception as e:
                    print(f"행 {i}의 책 링크 추출 실패: {e}")
                    continue
            
            if not page_links:
                print("현재 페이지에서 책 링크를 하나도 찾지 못했습니다. 종료합니다.")
                break
            
            # 각 책 링크를 순회하며 책 상세 페이지로 이동하고 데이터 수집
            for link in page_links:
                # href가 상대경로인 경우 기본 도메인 추가
                if link.startswith("/"):
                    link = "https://www.goodreads.com" + link
                print(f"책 링크 처리: {link}")
                self.driver.get(link)
                time.sleep(3)
                self.check_and_dismiss_overlay(timeout=5)
                # scrap_book()은 내부에서 해당 책의 상세 정보와 리뷰를 수집
                book_data = self.scrap_book()
                all_books.append(book_data)
            
            # 한 페이지의 모든 책을 처리한 후, 다음 페이지로 이동
            try:
                next_button = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='all_votes']/div[1]/a[12]"))
                )
                next_button.click()
                time.sleep(3)
                # 업데이트된 URL(혹은 그대로 사용)이 다음 페이지의 URL
                self.book_list_url = self.driver.current_url
                current_page += 1
            except Exception as e:
                print("다음 페이지 버튼 클릭 실패 또는 더 이상 페이지가 없습니다:", e)
                break
        
        self.output = all_books
        return all_books

    def close_browser(self):
        if self.driver:
            self.driver.quit()


# __main__ 코드: 책 리스트에서 첫 번째 책을 선택 후, 책 정보와 리뷰(영어 필터 적용)가 수집됨.
if __name__ == "__main__":
    output_directory = "./output"  
    # 결과를 저장할 폴더가 없으면 생성

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    crawler = BookInfoCrawler(output_directory)
    crawler.start_browser()
    
    book_data = crawler.scrap_list()
    print("수집된 책 데이터:")
    print(book_data)
    
    crawler.close_browser()


--- 책 리스트 페이지 1 처리 시작 ---
책 링크 처리: https://www.goodreads.com/book/show/2767052-the-hunger-games
로그인 오버레이가 해제되었습니다.
수집된 리뷰 1 (컨테이너 div[1]): {'reviewer': 'Nataliya', 'date': 'April 25, 2023', 'rating': 'Rating 4 out of 5', 'review': 'Suzanne Collins has balls ovaries of steel to make us willingly cheer for a teenage girl to kill other children. In a YA book.\nTwo reasons why this book rocks: (a) It is not Twilight, and (b) I really hate reality shows.\n\n\n\n\nYes, this book is full of imperfections. It often requires a strenuous suspension of disbelief. It can cause a painful amount of eye-rolling and shaking fist at the book pages. Its style is choppy and the first-person present tense gets annoying. The story is simple, and the message is heavy-handed. And here is an obligatory taken out of contest Twilight-bashing quote:\nBecause Katniss is cool and a badass. She is fierce, independent, resourceful, intelligent, and skilled. She is loyal to her friends and family. She is a survivor. 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00007FF79D0080D5+2992373]
	(No symbol) [0x00007FF79CC9BFD0]
	(No symbol) [0x00007FF79CB3590A]
	(No symbol) [0x00007FF79CB0F4F5]
	(No symbol) [0x00007FF79CBB63A7]
	(No symbol) [0x00007FF79CBCEE72]
	(No symbol) [0x00007FF79CBAF113]
	(No symbol) [0x00007FF79CB7A918]
	(No symbol) [0x00007FF79CB7BA81]
	GetHandleVerifier [0x00007FF79D066A2D+3379789]
	GetHandleVerifier [0x00007FF79D07C32D+3468109]
	GetHandleVerifier [0x00007FF79D070043+3418211]
	GetHandleVerifier [0x00007FF79CDFC78B+847787]
	(No symbol) [0x00007FF79CCA757F]
	(No symbol) [0x00007FF79CCA2FC4]
	(No symbol) [0x00007FF79CCA315D]
	(No symbol) [0x00007FF79CC92979]
	BaseThreadInitThunk [0x00007FF94496E8D7+23]
	RtlUserThreadStart [0x00007FF94557FBCC+44]
